In [3]:
import numpy as np
import pandas as pd

# Rotwein- und Weißweindaten einlesen
training = pd.read_csv('titanic_training.csv', sep = ",")


In [4]:
training.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
cor = training.corr()
print(cor)

             PassengerId  Survived    Pclass       Age     SibSp     Parch  \
PassengerId     1.000000 -0.005007 -0.035144  0.036847 -0.057527 -0.001652   
Survived       -0.005007  1.000000 -0.338481 -0.077221 -0.035322  0.081629   
Pclass         -0.035144 -0.338481  1.000000 -0.369226  0.083081  0.018443   
Age             0.036847 -0.077221 -0.369226  1.000000 -0.308247 -0.189119   
SibSp          -0.057527 -0.035322  0.083081 -0.308247  1.000000  0.414838   
Parch          -0.001652  0.081629  0.018443 -0.189119  0.414838  1.000000   
Fare            0.012658  0.257307 -0.549500  0.096067  0.159651  0.216225   

                 Fare  
PassengerId  0.012658  
Survived     0.257307  
Pclass      -0.549500  
Age          0.096067  
SibSp        0.159651  
Parch        0.216225  
Fare         1.000000  


In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
# layers = Schichten im NN-Modell
# Dense = Neuronen im Modell
# Activation = Aktivierungregel (wie sigmoid)
# Dropout = overfitting verhindern

#Netz definieren
model = Sequential()
model.add(Dense(5, activation = '')) # activation checken
# Schichten adden
model.add(Dropout(0.5)) 
model.add(Dense(1, activation = sigmoid())) # sigmoid import

#Compile
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) #adam checken 

#Train
train = model.fit(training[....], training[survied], epochs = 3, batch_size = 32, validation_split = 0.2) #epochs niedrig anfangen, validation_split 20 bis 30
val = np.mean(train.history['val_acc'])
print(val)

ModuleNotFoundError: No module named 'keras'